# 磁力線追跡

##### モジュールのインポートとRunge-Kuttaに渡す関数の定義

In [ ]:
import sub.plot as pl
import sub.functions as sb
import sub.electromagnetics as sel
from global_variables import gparam
import numpy as np
import vessel.vmat as svm
import sub.runge_kutta as ruke
import pandas as pd

gl = gparam()

In [ ]:
# Runge-Kuttaに渡す磁場の関数の定義
class Rmag(sel.Magnetic):
    direction = 1
    def get_val(self, t0, x0):
        b = np.array(self.get_mag(x0))
        b0 = b/np.sqrt(np.sum(b*b)) # normalization
        return b0*self.direction
    def is_end(self, t0, x0):
        x, y, z = x0
        r = np.sqrt(x**2+y**2)
        return not svm.is_inside_vac(r, z)

##### 磁場の計算条件の定義

In [ ]:
condition = {
    # TF current
    'cur_tf':{'tf': +50.0e+3, 'turn': 16},
    
    # initial plasma profile
    'cur_ip':{'ip':+100.0e+3, 'r0':0.65, 'z0':0.0, 'radius':0.3},
    
    # PF currents
    'cur_pf':{'pf17t12':-0.0e+3, 'pf26t36':-0.5e+3,'pf4_ab_cc2':-6.0e+3, 'pf35_1':-5.0e+3, 'pf35_2':-5.0e+3 },
    #'cur_pf':{'pf17t12':-1.0e+3, 'pf26t36':-1.0e+3,'pf4_ab_cc2':-0.0e+3, },
        
    # number of coefficients
    'num_dpr':1, # dp/df
    'num_di2':1, # di2/df
    
    'resolution': gl.get_dmat_coarse(),
    }

##### 磁場の計算と計算結果の確認

In [ ]:
# 磁場の計算, 平衡の計算
cond = sb.equi_pre_process(condition, verbose=0)
#cond = sb.calc_equilibrium(condition, verbose=0) # 平衡計算した後の磁力線長を追いたい場合はこちらを有効にする。

In [ ]:
# 計算結果の確認
pl.d_heatmap(cond['vessel']) # 真空容器（計算領域)
pl.d_contour(cond['flux_coil']) # flux分布, プラズマを含まないfluxはflux_coilで得られる。

##### 磁力線の座標を返す関数の定義

In [ ]:
# Runge-Kuttaに渡す関数の作成
mag = Rmag(cond['flux_coil'], cond['pol_current'])

In [ ]:
# 磁力線の座標を返す関数の定義
def calc_mag_trace(initial_point, step_width=0.01, step_num=10000):
    """磁力線の座標を返す。
    磁力線の座標間の距離がstep_widthになるので、
    磁力線の長さは、(出力された点の個数) X step_widthになります。

    Args:
        initial_point (array): [x, y, z]
        step_width (float): ステップ幅
        step_num (int): イタレーション回数
    
    Returns:
        array of [x,y,z]: poins of magnatic filed line
    """
    p0 = np.array(initial_point)
    crk = ruke.Runge_kutta(mag)
    # 開始点からプラス方向に探索
    mag.direction = +1
    pp = crk.solve(0, p0, step_width, step_num)
    # 開始点からマイナス方向に探索
    mag.direction = -1
    pn = crk.solve(0, p0, step_width, step_num)
    pts = np.vstack((pn[1:][::-1], pp)) # pn[1:][::-1] 重複する最初のデータを削除して反転
    return pts

##### 磁力線の座標の計算と計算結果の確認

In [ ]:
# 磁力線の座標の計算
# 磁力線追跡をする開始点[x, y, z]を指定してください。
i_point = [0.550, 0.0, -0.95] 
# 1回の計算での移動距離を指定してください。
s_width=0.01 
# step_numは最大の計算回数です。真空容器に到達する場合は、その時点で計算終了です。
pts = calc_mag_trace(i_point, s_width, step_num=6000) 
print(f'磁力線長さ：{len(pts)*s_width} m')

In [ ]:
pl.line_plot3d(pts) # 3D表示
pl.line_plot(pts) # ポロイダル断面への投影

##### CHI電極における磁力線長

- CHI電極はz < -1の領域にあるので、計算領域を広げる必要があります。  
  global_variables.pyのcname = "f0"の３行を無効にして、cname = "f1"の３行を有効に指定ください。
- 真空容器の領域も修正する必要があります。  
  /vessel/vmat.pyでCHI電極用の真空容器の範囲にしてください。

In [ ]:
# 計算する開始点の定義
#pts = [(0.389, 0, e) for e in np.linspace(-1.305,-1.165,10)]
#p0 = [(e, 0, -1.165) for e in np.linspace(0.389,0.564,10)]
#pts.extend(p0[1:])

In [ ]:
# 磁力線追跡
#s_width = 0.01
#dat = [calc_mag_trace(i_point, s_width, step_num=6000) for i_point in pts]

In [ ]:
# トータルの磁力線長
def meas_tot(p):
    return len(p)*s_width

# ポロイダル距離
def meas_pol(p):
    """_summary_

    Args:
        p (_type_): _description_

    Returns:
        _type_: _description_
    """
    p0 = np.array(p)
    a = [(np.sqrt(x**2+y**2), z) for x, y, z in p0] # (x, y, z)を(r, z)に変換
    a = np.array(a)
    b = np.diff(a, axis=0) # 差分を取り(dr, dz)を計算
    c = [np.sqrt(dr**2+dz**2) for dr, dz in b] # 距離を計算
    return sum(c)

# トロイダル距離
def meas_tor(p):
    p0 = np.array(p)
    a = [(x, y) for x, y, z in p0] # (x, y, z)を(x, y)に変換
    a = np.array(a)
    b = np.diff(a, axis=0) # 差分を取り(dx, dy)を計算
    c = [np.sqrt(dx**2 + dy**2) for dx, dy in b] # 距離を計算
    return sum(c)

#回転回数を計算する 
def meas_rot(e):         
    dif = np.diff(e,axis=0)
    d = [(np.sqrt(e[0]**2 + e[1]**2)) for e in dif]
    rz = [(np.sqrt(x**2 + y ** 2),z) for x,y,z in e]                                                                                           
    rad_diff = [d/rz[0] for (d,rz) in zip(d,rz)]
    rad = np.sum(rad_diff)
    rot = rad/ (np.pi * 2)
    
    return rot
    

In [ ]:
#meas = [(i, meas_tot(e), meas_tor(e), meas_pol(e),meas_rot(e)) for i, e in enumerate(dat)]

In [ ]:
#df = pd.DataFrame(meas, columns=['num', 'total', 'toroidal', 'poloidal','rotation'])
#df.set_index('num', inplace=True)

In [ ]:
#df.to_csv('distance.csv')

In [ ]:
#pl.iplot_df(df)